In [392]:
import os
import sys
import tqdm
import random
from dataclasses import dataclass, asdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

### Settings

In [393]:
@dataclass
class Config:
    FILE_NAME: str = 'TCN'
    ROOT_DIR: str = './data/huge'
    SEED: int = 42
    
    DEVICE: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    BATCH_SIZE: int = 128
    EPOCHS: int = 30
    LEARNING_RATE: float = 0.00005
    SERIES_LENGTH: int = 50
    
    KERNEL_SIZE: int = 4
    NUM_CHANNELS = [128, 128, 128, 128]
    DROPOUT: float = 0.2
    OUTPUT_DIM: int = 1
    
CONFIG = Config()

In [394]:
seed = CONFIG.SEED
np.random.seed(seed)
torch.manual_seed(seed)

In [395]:
import mlflow
import mlflow.pytorch

mlflow.set_experiment('SKADA')
def mlflow_run_decorator(run_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            mlflow.start_run(run_name=run_name)
            try:
                result = func(*args, **kwargs)
                mlflow.set_tag("Status", "SUCCEESS")
            except Exception as e:
                mlflow.log_param("Exception", e)
                mlflow.set_tag("Status", "FAIL")
                raise e
            finally:
                mlflow.end_run()
            return result
        return wrapper
    return decorator

### Data

In [396]:
train_df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_df.csv"), index_col='Serial Number')
train_y = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_y.csv"), index_col='Serial Number')
pred_x = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "pred_x.csv"), index_col='Serial Number')

In [397]:
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

pred_x['TIMESTAMP'] = pd.to_datetime(pred_x['TIMESTAMP'])
pred_x['TIMESTAMP'] = pred_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

serial_key = list(train_df.columns)[0]
date_time_key = list(train_df.columns)[1]
feature_keys = list(train_df.columns)[3:-1]
target_key = list(train_df.columns)[-1]

train_x = train_df.drop(columns='Y')

In [398]:
print("Train Data 크기 :", train_x.shape)
print("Pred Data 크기 :", pred_x.shape)

print("Train Data의 Serial Number의 unique 값 :", len(train_x[serial_key].unique()))
print("Pred Data의 Serial Number의 unique 값 :", len(pred_x[serial_key].unique()))

Train Data 크기 : (555456, 19)
Pred Data 크기 : (138880, 19)
Train Data의 Serial Number의 unique 값 : 97860
Pred Data의 Serial Number의 unique 값 : 72562


In [399]:
train_x.head(3)

,TIMESTAMP,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,,
19,2020-02-09 16:24,EI83N072710203N8H,22,18,22,16,32,99,91219,43095,2132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-10 16:39,EI83N072710203N8H,22,18,22,16,32,99,91413,43214,2133,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-11 16:54,EI83N072710203N8H,22,18,22,16,32,99,91606,43332,2135,49.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


### PreProcessing

In [400]:
train_x.drop(columns='X1', inplace=True)
pred_x.drop(columns='X1', inplace=True)

In [401]:
scaler = StandardScaler()

train_x[feature_keys] = scaler.fit_transform(train_df[feature_keys])
pred_x[feature_keys] = scaler.transform(pred_x[feature_keys])

In [402]:
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
pred_x_by_serial = [group[1] for group in pred_x.groupby(pred_x.index)]

train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
pred_x_by_serial = [group.sort_values('TIMESTAMP') for group in pred_x_by_serial]

### Data

In [403]:
def train_val_test_split(Xs, ys, val_ratio=0.2, test_ratio=0.1):
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        data_per_label[label].append((x, y))

    train, val, test = [], [], []

    for label in data_per_label:
        data = data_per_label[label]
        n_test = int(len(data) * test_ratio)
        n_val = int(len(data) * val_ratio)
        test += data[:n_test]
        val += data[n_test:n_test+n_val]
        train += data[n_test+n_val:]

    X_train, y_train = zip(*train)
    X_val, y_val = zip(*val)
    X_test, y_test = zip(*test)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [404]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(train_x_by_serial, train_y['Y'])

X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in X_test]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 5791
Validation Data의 개수 : 1654
Test Data의 개수 : 827


In [405]:
def align_data(data, series_length):
    data_features = [x[feature_keys] for x in data]
    length_aligned_X = []
    
    for x in data_features:
        if len(x) >= series_length:
            length_aligned_X.append(x[-series_length:])
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X)

In [406]:
X_train = align_data(X_train, CONFIG.SERIES_LENGTH)
X_val = align_data(X_val, CONFIG.SERIES_LENGTH)
X_test = align_data(X_test, CONFIG.SERIES_LENGTH)

In [407]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

### Dataset

In [408]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32).permute(1,0), torch.tensor(self.y[idx], dtype=torch.float32)

In [409]:
train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [410]:
train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

### Model

In [411]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.net1 = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1)
        
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.net2 = nn.Sequential(self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net1(x)
        out = self.net2(out)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.conv_out = nn.Conv1d(num_channels[-1], 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.network(x)
        x = self.conv_out(x[:, :, -1:])
        x = x.squeeze(2)
        return self.sigmoid(x)

In [412]:
input_dim = len(feature_keys)
model = TemporalConvNet(input_dim, CONFIG.NUM_CHANNELS, kernel_size = CONFIG.KERNEL_SIZE, dropout = CONFIG.DROPOUT)
model = model.to(CONFIG.DEVICE)

In [413]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG.LEARNING_RATE)

### Training

In [414]:
@mlflow_run_decorator(run_name=CONFIG.FILE_NAME)
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    mlflow.log_params(asdict(CONFIG))
    
    device = CONFIG.DEVICE
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
        
        val_loss = 0.0
        model.eval()
        all_preds = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                
                val_loss += loss.item() * inputs.size(0)
                all_preds.extend(outputs.squeeze().cpu().numpy())
        
        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        
        val_preds = (np.array(all_preds) > 0.5).astype(int)
        f1 = f1_score(y_val, val_preds, average='macro')
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 Score: {f1:.4f}')
        mlflow.log_metric('Train Loss', train_loss, step=epoch)
        mlflow.log_metric('Val Loss', val_loss, step=epoch)
        mlflow.log_metric('Val F1 Score', f1, step=epoch)
    
    return model

In [415]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, CONFIG.EPOCHS)

Epoch 1/30, Train Loss: 0.6016, Val Loss: 0.4792, Val F1 Score: 0.4605
Epoch 2/30, Train Loss: 0.3465, Val Loss: 0.2714, Val F1 Score: 0.4605
Epoch 3/30, Train Loss: 0.1999, Val Loss: 0.1764, Val F1 Score: 0.8626
Epoch 4/30, Train Loss: 0.1445, Val Loss: 0.1547, Val F1 Score: 0.8800
Epoch 5/30, Train Loss: 0.1212, Val Loss: 0.1346, Val F1 Score: 0.8851
Epoch 6/30, Train Loss: 0.1084, Val Loss: 0.1282, Val F1 Score: 0.8875
Epoch 7/30, Train Loss: 0.1026, Val Loss: 0.1214, Val F1 Score: 0.8920
Epoch 8/30, Train Loss: 0.0972, Val Loss: 0.1198, Val F1 Score: 0.8966
Epoch 9/30, Train Loss: 0.0910, Val Loss: 0.1147, Val F1 Score: 0.8997
Epoch 10/30, Train Loss: 0.0919, Val Loss: 0.1179, Val F1 Score: 0.9044
Epoch 11/30, Train Loss: 0.0866, Val Loss: 0.1095, Val F1 Score: 0.9069
Epoch 12/30, Train Loss: 0.0841, Val Loss: 0.1023, Val F1 Score: 0.9105
Epoch 13/30, Train Loss: 0.0823, Val Loss: 0.1114, Val F1 Score: 0.9118
Epoch 14/30, Train Loss: 0.0797, Val Loss: 0.1060, Val F1 Score: 0.9153
E

In [416]:
model.eval()
test_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(CONFIG.DEVICE)
        outputs = model(inputs)
        
        test_preds.extend(outputs.squeeze().cpu().numpy())

test_preds = (np.array(test_preds) > 0.5).astype(int)
f1 = f1_score(y_test, test_preds, average='macro')
print(f'Test 데이터에 대한 F1 Score: {f1:.8f}')

Test 데이터에 대한 F1 Score: 0.87748148


### Pred

In [391]:
submission = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "test_y.csv"), index_col='Serial Number')

y_test_pred = clf.predict(X_test)
submission["Y"] = y_test_pred
submission.to_csv("submission.csv", index_label='Serial Number')

FileNotFoundError: [Errno 2] No such file or directory: './data/huge\\test_y.csv'

In [ ]:
submission = pd.read_csv("submission.csv", index_col='Serial Number')
submission

In [ ]:
print(submission["Y"].mean())
print(train_y.mean())

Public Score : 0.8130563798219584